In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型花园 - StarCoder

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_starcoder.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_starcoder.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_starcoder.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在Vertex AI Workbench中打开
    </a>
    （建议使用Python-3 CPU笔记本）
  </td>
</table>

## 概述

本笔记本演示了在Vertex AI上部署预训练的[StarCoder](https://huggingface.co/bigcode/starcoder)模型进行在线预测。

### 目标

- 将模型上传到[模型注册表](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)。
- 在[端点](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints)上部署模型。
- 运行用于代码生成的在线预测。

### 成本

本教程使用Google Cloud的收费组件：

* Vertex AI

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

## 设置环境

**注意**: Jupyter会将以`!`为前缀的行作为shell命令运行，并将以`$`为前缀的Python变量插入这些命令中。

只有协作

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### 设置谷歌云项目

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建一个账户时，您会获得 $300 的免费信用额用于您的计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API 和 Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

4. [创建一个 Cloud Storage 存储桶](https://cloud.google.com/storage/docs/creating-buckets) 用于存储实验输出。

填写以下变量以进行实验环境设置：

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = "us-central1"  # @param {type:"string"}

初始化Vertex AI API

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

### 设置 Hugging Face 访问令牌

StarCoder 模型需要一个访问令牌。

1. [登录或创建 Hugging Face 账户](https://huggingface.co/login)。

2. [在 Bigcode/Starcoder 页面接受模型许可协议](https://huggingface.co/bigcode/starcoder)。

3. [创建一个访问令牌](https://huggingface.co/settings/tokens)，具有 "read" 角色。访问令牌将以 `"hf_"` 开头的字符串形式呈现。更多详情，请查看 [说明页面](https://huggingface.co/docs/hub/security-tokens)。

4. 填写下方的访问令牌：

In [ ]:
# Hugging Face access token.
ACCESS_TOKEN = ""  # @param {type:"string"}

### 定义常量

In [ ]:
# The pre-built serving docker image.
# The model artifacts are embedded within the container, except for model weights which will be downloaded during deployment.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-transformers-serve"

### 定义常见函数

In [ ]:
def deploy_model(model_id, access_token):
    """Uploads and deploys the model to Vertex AI endpoint for prediction."""
    model_name = "starcoder"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "ACCESS_TOKEN": access_token,
        "DEPLOY_SOURCE": "notebook",
    }
    # If the model_id is a GCS path, use artifact_uri to pass it to serving docker.
    artifact_uri = model_id if model_id.startswith("gs://") else None
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/transformers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
        artifact_uri=artifact_uri,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type="n1-standard-8",
        accelerator_type="NVIDIA_TESLA_V100",
        accelerator_count=4,
        deploy_request_timeout=3600,
    )
    return model, endpoint

## 上传并部署模型

这一部分将预训练模型上传到模型注册表，并在具有4个V100 GPU的端点上部署。

模型部署步骤大约需要25分钟才能完成。

In [ ]:
model, endpoint = deploy_model(model_id="bigcode/starcoder", access_token=ACCESS_TOKEN)

注意：模型权重将在部署成功后下载。因此，在上述模型部署步骤成功之后，需要额外等待20分钟，然后再运行下面的下一步。否则，在向端点发送请求时，可能会出现`ServiceUnavailable:503 502:Bad Gateway`错误。

发送预测请求

In [ ]:
input_text = "def print_hello_world():"

instances = [
    {"text": input_text},
]
preds = endpoint.predict(instances=instances).predictions
print(preds[0].get("generated_text"))

清理

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()